In [1]:
# Load the Models

import pickle
import json
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
# Read the Files

model_ann=load_model('m1/churn_model.tf', compile = False)

with open('m1/imputer_num.pkl', 'rb') as file_1:
    imputer_num=pickle.load(file_1)
    
with open('m1/imputer_cat.pkl', 'rb') as file_m1:
    imputer_cat=pickle.load(file_m1)
    
with open('m1/windsoriser.pkl', 'rb') as file_4:
    windsoriser=pickle.load(file_4)
    
with open('m1/preprocessor.pkl', 'rb') as file_9:
    preprocessor=pickle.load(file_9)  

with open('m1/num_cols.txt', 'r') as file_5:
    num_cols=json.load(file_5)

with open('m1/cat_cols.txt','r') as file_6: 
    cat_cols=json.load(file_6)


In [3]:
# Create New Data

data_inf = {
    'user_id' : '8590af72adf4f3a9',
    'age' : 52,
    'gender' : 'F',
    'region_category' : 'City',
    'membership_category' : 'No Membership',
    'joining_date' : '2016-05-16',
    'joined_through_referral' : 'Yes',
    'preferred_offer_types' :'Credit/Debit Card Offers',
    'medium_of_operation':np.nan,
    'internet_option':'Mobile_Data',
    'last_visit_time':'11:11:03',
    'days_since_last_login':11,
    'avg_time_spent':301.67,
    'avg_transaction_value':14545.56,
    'avg_frequency_login_days':16.0,
    'points_in_wallet':0.0,
    'used_special_discount':'No',
    'offer_application_preference':'Yes',
    'past_complaint':'Yes',
    'complaint_status':'Solved',
    'feedback':'Too many ads'
}

data_inf = pd.DataFrame([data_inf])
data_inf

,user_id,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,8590af72adf4f3a9,52,F,City,No Membership,2016-05-16,Yes,Credit/Debit Card Offers,NaN,Mobile_Data,...,11,301.67,14545.56,16.0,0.0,No,Yes,Yes,Solved,Too many ads


In [15]:
num_cols

['age',
 'days_since_last_login',
 'avg_time_spent',
 'avg_transaction_value',
 'avg_frequency_login_days',
 'points_in_wallet']

In [4]:
# Split X data based on data type
X_test_num = data_inf[num_cols]
X_test_cat = data_inf[cat_cols]

In [5]:
# Handling Missing Value
X_test_num=pd.DataFrame(imputer_num.transform(X_test_num), columns=num_cols)
X_test_cat=pd.DataFrame(imputer_cat.transform(X_test_cat), columns=cat_cols)

In [6]:
# Handling Outlier
X_test_num_capped = windsoriser.transform(X_test_num)

In [7]:
# Feature Selection
X_test_num_capped.drop(['age'], axis=1, inplace=True)
X_test_cat.drop(['past_complaint', 'complaint_status', 'last_visit_time', 'user_id'], axis=1, inplace=True)

In [8]:
# Concat all X data
X_test_concat=pd.concat([X_test_num_capped.reset_index(drop=True),X_test_cat.reset_index(drop=True)], axis=1)

In [9]:
# Transform X data using column transformer pipeline (scaling and encoding)
X_test_final=preprocessor.transform(X_test_concat)

In [14]:
# Model Evaluation of test-set

y_pred_seq2 = model_ann.predict(X_test_final)
y_pred_seq2 = np.where(y_pred_seq2 >= 0.5, 1, 0)
if y_pred_seq2==1:
    print ("The customer will be churning")
else:
    print ("The customer will not be churning")

1/1 [==============================] - 0s 83ms/step
The customer will be churning


Diprediksikan bahwa customer baru kita akan melakukan churn, menurut model ANN yang kita pakai.